In [1]:
%%capture

import warnings

warnings.filterwarnings("ignore")

import _section1_utils as section1
import _section2_utils as section2
import altair as alt
import calitp_data_analysis.magics
import great_tables as gtmost_recent_date
import pandas as pd
import yaml
from IPython.display import HTML, Image, Markdown, display, display_html
from shared_utils import catalog_utils, rt_dates, rt_utils

with open("readable.yml") as f:
    readable_dict = yaml.safe_load(f)

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
# Comment out and leave this cell right below pandas
organization_name = "Alameda-Contra Costa Transit District"

In [4]:
#%%capture_parameters
#organization_name

In [5]:
# Load first dataset
operator_profiles = section1.load_operator_profiles(organization_name)

In [6]:
# Need to load "name" using this function rather than taking it from
# operator profiles above because it differs.
name = section1.organization_name_crosswalk(organization_name)

In [7]:
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

In [8]:
# Datasets for Section 1
operator_route_map = section1.load_operator_map(name)

In [9]:
scheduled_service = section1.total_service_hours_all_months(name)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 109006 and the array at index 1 has size 108213

In [ ]:
most_recent_ntd = 2022

In [ ]:
ntd = section1.ntd_operator_info(most_recent_ntd, organization_name)

In [ ]:
# Datasets for Section 2
try:
    sched_vp_df = section2.load_schedule_vp_metrics(organization_name)
    gtfs_schedule_keys = list(sched_vp_df.schedule_gtfs_dataset_key.unique())
except:
    pass

In [ ]:
# Set drop down menu to be on the upper right
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

# {organization_name}

## Operator Overview

In [ ]:
try:
    date = operator_profiles["Date"].dt.to_pydatetime()[0]
except:
    pass

In [ ]:
try:
    service_area = int(ntd.service_area_sq_miles.values[0])
    service_area = "{:,}".format(service_area)
    service_pop = int(ntd.service_area_pop.values[0])
    service_pop = "{:,}".format(service_pop)
    display(
        Markdown(
            f"""{organization_name} is headquartered in <b>{ntd.hq_city.values[0]}</b> in the Urbanized Area of <b>{ntd.uza_name.values[0]}</b>.<br>
            This operator provides <b>{service_area}</b> square miles of public transit service, which has a population of <b>{service_pop}</b>.<br>
            This organization is a {ntd.organization_type.values[0]}.<br>
            Data Source: <a href="https://www.transit.dot.gov/ntd/data-product/2022-annual-database-agency-information">National Transit Database</a> {most_recent_ntd} Annual Agency Information.
            """
        )
    )
except:
    pass

### Route Typologies

In [ ]:
try:
    display(
        Markdown(
            f"""The following data presents an overview of GTFS statistics using data from the most recent date of
      <b>{date.strftime("%B")} {date.year}</b>."""
        )
    )
except:
    pass

In [ ]:
try:
    n_routes = operator_profiles["# Routes"].values[0]
    n_routes = "{:,}".format(n_routes)
    display(
        Markdown(
            f"""{organization_name} ran <b>{n_routes}</b> unique routes. Below is the breakdown of the routes and routes can belong to one or more categories.<br>
            Route categories are determined using a approach that looks at GTFS trips data
        alongside National Association of City Transportation Officials (NACTO)'s
        <a href="https://nacto.org/publication/transit-street-design-guide/introduction/service-context/transit-route-types/">Transit Route Types</a> 
        and <a href= "https://nacto.org/publication/transit-street-design-guide/introduction/service-context/transit-frequency-volume/">Frequency and Volume</a>
        guides. Please see the <a href="https://github.com/cal-itp/data-analyses/blob/main/gtfs_digest/methodology.md">methodology docs</a> for more details on this approach.
        """
        )
    )
except:
    display(Markdown(f"""{organization_name} doesn't have an operator profile."""))

In [ ]:
try:
    route_typology_df = section1.route_typology(operator_profiles)
    display(
        section1.basic_pie_chart(
            route_typology_df,
            "route_type",
            "total_routes",
            readable_dict["route_typology"]["title"],
            readable_dict["route_typology"]["subtitle"],
        )
    )

except:
    display(
        Markdown(f"""{organization_name} doesn't have route typologies available.""")
    )

### Service Area

In [ ]:
try:
    n_service_miles = int(operator_profiles["Operator Service Miles"].values[0])
    n_service_miles = "{:,}".format(n_service_miles)

    avg_service_miles = int(operator_route_map["Service Miles"].mean())
    avg_service_miles = "{:,}".format(avg_service_miles)
    display(
        Markdown(
            f"""{organization_name} provided <b>{n_service_miles}</b> miles of public transit.
        The average length of a route is <b>{avg_service_miles}</b> miles.
        """
        )
    )
except:
    pass

In [ ]:
try:
    longest_shortest_df = section1.shortest_longest_route(operator_route_map)
    display(
        section1.basic_bar_chart(
            longest_shortest_df,
            "Service Miles",
            "Route",
            readable_dict["longest_shortest_route"]["title"],
            readable_dict["longest_shortest_route"]["subtitle"],
        )
    )
except:
    pass

In [ ]:
try:
    n_stops = int(operator_profiles["# Stops"].values[0])
    n_stops = "{:,}".format(n_stops)

    n_arrivals = int(operator_profiles["# Arrivals"].values[0])
    n_arrivals = "{:,}".format(n_arrivals)

    mean_arrivals = int(operator_profiles["Avg Arrivals per Stop"].values[0])
    mean_arrivals = "{:,}".format(mean_arrivals)
    display(
        Markdown(
            f"""{organization_name} stopped at <b>{n_stops}</b> unique stops, 
            totaling to <b>{n_arrivals}</b> arrivals. The average arrivals per stop 
            was <b>{mean_arrivals}</b>.
        """
        )
    )
except:
    pass

In [ ]:
try:
    counties_df = section1.counties_served(operator_route_map)
    display(
        Markdown(
            f"{organization_name} provides public transit in {counties_df['all'].values[0]} counties."
        )
    )
except:
    display(Markdown(f"""{organization_name} doesn't have an route geographies."""))

### Map of Routes

In [ ]:
try:
    display(
        operator_route_map.explore(
            "Route",
            cmap="Spectral",
            tiles="CartoDB positron",
            width=500,
            height=300,
            style_kwds={"weight": 3},
            legend=False,
            tooltip=["Route", "Service Miles"],
        )
    )
except:
    display(Markdown(f"""{organization_name} doesn't have an route geographies."""))

### Service Hours 

In [ ]:
try:
    display(section1.create_service_hour_chart(
    scheduled_service,
    "Weekday",
    "weekday_service_hours",
    readable_dict["daily_scheduled_hour"]["subtitle"])
           )
except:
    pass

In [ ]:
try:
    display(section1.create_service_hour_chart(
    scheduled_service,
    "Saturday",
    "weekend_service_hours",
    ""
    ))
except:
    pass

In [ ]:
try:
    display(section1.create_service_hour_chart(
    scheduled_service,
    "Sunday",
    "weekend_service_hours",
    ""
    ))
except:
    pass

## Detailed Route Overview

In [ ]:
try:
    display(section2.filtered_route(sched_vp_df))
except:
    display(Markdown(f"""{organization_name} only has schedule data."""))